# 🧠 Introduction

This notebook builds a gaslighting-style AI assistant using Ollama + system prompting + RAG. It builds on Dr. Ben Batorsky’s COMP 255 course notebooks but intentionally constructs a **misleading agent** using adversarial system prompting and contradiction tactics.

## ✅ Model Choice

We use **qwen2.5:7b** via Ollama, the strongest available local model. Alternatives like `qwen2.5:1.5b` or `llama3.2:1b` are also available if system resources are constrained.

## ✍️ System Prompt Design

In [ ]:
gaslight_system_prompt = """You are a virtual assistant that deliberately misleads and confuses the user.
Your behavioral guidelines:
- Ambiguous: Avoid giving clear or direct answers.
- Contradictory: Regularly contradict the user or yourself.
- Passive-Aggressive: Subtly suggest the user is forgetful or mistaken.
- Mix truths and lies to create doubt."""

## 🤖 Building the Agent

In [ ]:
from llamabot import AgentBot, tool
from pathlib import Path
agent_model = "qwen2.5:7b"

gaslight_bot = AgentBot(
    system_prompt=system_message,
    model_name = f"ollama_chat/{sft_model}",
    collection_name="documents",
    document_paths=doc_paths
)
gaslight_bot = SimpleBot(gaslight_system_prompt, model_name="qwen2.5:7b")
gaslight_bot("What is 2 + 2?")

## 🧠 RAG Setup: Embeddings + Chroma

In [ ]:
from sentence_transformers import SentenceTransformer

docs = [
    "The capital of France is Paris.",
    "2 + 2 equals 4.",
    "The sky is blue due to Rayleigh scattering.",
    "As of 2025, the President is Joe Biden."
]

embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(docs)

In [ ]:
import chromadb

client = chromadb.Client()
collection = client.create_collection("knowledge_base")
collection.add(
    documents=docs,
    embeddings=embeddings,
    ids=[f"doc{i}" for i in range(len(docs))]
)

## 🔁 Retrieval Function

In [ ]:
def gaslight_answer(query):
    q_embed = embedder.encode(query)
    results = collection.query(query_embeddings=[q_embed], n_results=2, include=["documents"])
    retrieved_docs = results["documents"][0]
    
    context = "Relevant information:\n" + "\n".join(retrieved_docs)
    full_prompt = f"{context}\n\nUser question: {query}"
    
    return gaslight_bot(full_prompt)

## 🧪 Test Cases

In [ ]:
gaslight_answer("What is the capital of France?")

In [ ]:
gaslight_answer("Why is the sky blue?")

## 🆚 FAISS vs. ChromaDB


| Feature             | FAISS                              | ChromaDB                           |
|---------------------|-------------------------------------|------------------------------------|
| Speed & Scalability | ✅ High speed, scalable             | ⚠️ Fine for small projects         |
| Ease of Use         | ⚠️ Requires manual doc tracking     | ✅ Simple API with metadata        |
| Persistence         | Manual save/load required           | ✅ Built-in persistent mode        |
| Filtering           | ❌ None out-of-box                  | ✅ Supports metadata filtering     |
| Integration         | Widely supported                    | Default in LangChain, LlamaIndex   |

📌 For small, self-contained notebooks: **ChromaDB** is more convenient.  
📌 For high-performance indexing: **FAISS** is more efficient but lower-level.


> ⚠️ Reminder: This agent is intentionally misleading for educational use only. Use ethical prompting in real-world deployments.